# Experimentation

In [2124]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import networkx as nx
from parse import *
from utils  import *
from pulp import *
from math import comb
import gurobipy as gp
from gurobipy import GRB

In [2125]:
sample_input = pd.read_csv("samples/10.in", header = None)
sample_output = np.array(pd.read_csv("samples/10.out", header = None))

In [2126]:
graph, budget = read_input_file("samples/10.in")

In [2127]:
pos = nx.kamada_kawai_layout(graph)  # positions for all nodes
# nx.draw(graph, pos, with_labels=True)

In [2128]:
# prob = LpProblem("proj",LpMaximize)

In [2129]:
# # edges 
# # Add diagonal variables such that (1,1) have 0 happiness and sadness
# edges = np.array(["e01","e02","e03","e12","e13","e23"])
# weights = [9.2, 5.4, 2.123, 75.4,18,87]*3
# sad = [3,40.8,98,8,57.904,9.4]*3
# d = {}
# for j in range(3):
#     for i in edges:
#         d["{0}_{1}".format(i,j)] = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# edges = list(d.values())
# # e11 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# # e22 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# # e33 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")

In [2130]:
# # vertices 
# vertices = ["v0", "v1", "v2", "v3"]
# d = {}
# for j in range(3):
#     for i in vertices:
#         d["{0}_{1}".format(i,j)] = LpVariable("{0}_{1}".format(i,j), lowBound = 0, cat="Integer")
# vertices = list(d.values())
# vertices

In [2131]:
# prob += sum(edges[i] * weights[i] for i in range(len(weights)))

In [2132]:
# prob += sum(edges[i] for i in [0,6,12]) == 1
# prob += sum(edges[i] for i in [1,7,13]) == 1
# prob += sum(edges[i] for i in [2,8,14]) == 1
# prob += sum(edges[i] for i in [3,9,15]) == 1
# prob += sum(edges[i] for i in [4,10,16]) == 1
# prob += sum(edges[i] for i in [5,11,17]) == 1

In [2133]:
# prob += sum(edges[i] * sad[i] for i in range(6)) <= 42.314/3
# prob += sum(edges[i] * sad[i] for i in range(6,12)) <= 42.314/3
# prob += sum(edges[i] * sad[i] for i in range(12,18)) <= 42.314/3

# prob += sum(vertices[i] for i in [0,4,8]) == 1 
# prob += sum(vertices[i] for i in [1,5,9]) == 1
# prob += sum(vertices[i] for i in [2,6,10]) == 1
# prob += sum(vertices[i] for i in [3,7,11]) == 1


# # Linearization of quadratic variables (eijk =vik * vjk)

In [2134]:
# prob

In [2135]:
# prob.solve()
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# Scale  

Assumptions 
- Rooms with individuals are usually not optimal and hence are not considered in the model
- We have a good heuristic for choosing the number of rooms

<!-- - Either $e_{i,i} = 1$ or $e_{i,j} = 1, \ \ \ \exists j$ -->
- Pick R arbitrarily 
    - Work on heuristics later
<!-- - Add $e_{i,i} \forall \ \ i$ -->
- Create k * (n choose 2) variables representing each pair and its class i.e,  e_i,j,k  for $k \in \{2, n-1\}$
- Objective $max\{\sum e_{i,j,k}*h_{i,j}\}$ 

- Constraints
    - $\sum_{k = 1}^{K} e_{i,j,k} = 1  \ \ \ \ \forall (i,j)$
    - If $(i,j) = 1 \And \ (i,h) = 1 \implies (j,h) = 1$
         - In LP notation: 
    - $\sum_{(i,j)} s_{i,j}e_{i,j,k} \leq \frac{S_{max}}{K} \ \ \ \forall k \in K$ 

- Parse the inputs into H and S arrays representing happiness and sadness 
- Get K number of rooms (improve later)
- Define edges and vertices as LpVariables 
    - Make them into two arrays, one sorted based on ids and another based on rooms (4 arrays in total)
- Define constraints 

In [2136]:
n = int(list(sample_input[0])[0])
s_max = int(list(sample_input[0])[1])
K = 3

In [2137]:
# Parsing input data 
df = sample_input
df = pd.DataFrame([x.split() for x in df[0].tolist() ])
df = df.loc[2:,:]
df["i"] = df.loc[:,0]
df["j"] = df.loc[:,1]
df["sadness"] = df.loc[:,3]
df["happiness"] = df.loc[:,2]
df = df[["i", "j", "sadness", "happiness"]]

In [2138]:
i_id = [int(i) for i in list(df["i"])]
j_id = [int(i) for i in list(df["j"])]
sadness = [float(i) for i in list(df["sadness"])]*K
happiness = [float(i) for i in list(df["happiness"])]*K

sadness_1 = [float(i) for i in list(df["sadness"])]
happiness_1 = [float(i) for i in list(df["happiness"])]
len(sadness) == len(happiness)

True

In [2139]:
prob = LpProblem("solver",LpMaximize)

In [2140]:
# edges 
d_e = {}
for k in range(K):
    for i in range(comb(n,2)):
        d_e["e_{0}_{1}_{2}".format(i_id[i],j_id[i],k)] = LpVariable("e_{0}_{1}_{2}".format(i_id[i],j_id[i],k), cat="Binary")
edges_1 = list(d_e.values())


# Create another edge array with different sort
edges_2 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[4]))]

# Create another edge array for constraint 4 
edges_3 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[6]))]

# Check
len(edges_1) == comb(n,2)*K

True

In [2141]:
# vertices 
d_v = {}
for j in range(K):
    for i in range(n):
        d_v["v_{0}_{1}".format(i,j)] = LpVariable("v_{0}_{1}".format(i,j), cat="Binary")
vertices_1 = list(d_v.values())

# Create another edge array with different sort
vertices_2 = [d_v.get(i) for i in sorted(d_v.keys(), key=lambda x: x[2])]

# Check
len(vertices_1) == n*K

True

In [2142]:
# Linearization of quadratic variables 1 
v_list = np.array(np.array_split(vertices_1, 3)).tolist()
v_big = []
for i in range(K):
    j = 0
    while j != n:
        for _ in range(j,9):
            v_big.append(v_list[i][j])
        j += 1
        
print(len(edges_1) == len(v_big))

True


In [2143]:
# Linearization of quadratic variables 2
v_list_2 = [i[1:] for i in v_list]
v_big_2 = []
for h in range(K):
    j = 0
    while j != n:
        for i in range(j,n-1):
            v_big_2.append(v_list_2[h][i])
        j += 1
        i = j
print(len(edges_1) == len(v_big_2))

True


## Solver 

In [2144]:
# Objective function 
prob += sum(edges_1[i] * happiness[i] for i in range(len(happiness)))

In [2145]:
# # Not sure if we need this constraint now 
# # Constraint 1 
# constraint_2 = [sum(edges_1[i:i + K]) for i in range(0, len(edges_1), K)]
# for i in range(len(constraint_3)):
#     prob += constraint_2[i] == 1

In [2146]:
# Constraint 2
# Each rooms has sadness levels <= s_max / K 
aa = []
h = 0
for i in range(K):
    for j in range(comb(n,2)):
        aa.append(edges_1[h] * sadness_1[j])
        h += 1

constraint_2 = [sum(aa[i:i+ comb(n,2)]) for i in range(0, len(aa), comb(n,2))]
for i in range(len(constraint_2)):
    prob += constraint_2[i] <= s_max / K

In [2147]:
# Constraint 3
# Each person belongs to only one room
constraint_3 = [sum(vertices_2[i:i + K]) for i in range(0, len(vertices_2), K)]
for i in range(len(constraint_3)):
    prob += constraint_3[i] == 1

In [2148]:
# Linearization of quadratic variables
# Constraint 4.1

for i in range(len(edges_1)):
    prob += edges_1[i] <= v_big[i]
    
# Constraint 4.2
for i in range(len(edges_1)):
    prob += edges_1[i] <= v_big_2[i]


# Constraint 4.3
for i in range(len(edges_1)):
    prob += edges_1[i] >= v_big[i] + v_big_2[i] - 1 
    
# # Constraint 4'
# for i in range(len(edges_1)):
#     prob += edges_1[i] == v_big[i] * v_big_2[i]

In [2149]:
path = "/Users/smeetpatel/Library/gurobi.lic"
solver = GUROBI_CMD(path)

In [2150]:
prob.solve(solver)
for v in prob.variables():
    print(v.name, "=", v.varValue)

e_0_1_0 = 0.0
e_0_1_1 = 0.0
e_0_1_2 = 0.0
e_0_2_0 = 0.0
e_0_2_1 = 1.0
e_0_2_2 = 0.0
e_0_3_0 = 0.0
e_0_3_1 = 0.0
e_0_3_2 = 0.0
e_0_4_0 = 0.0
e_0_4_1 = 0.0
e_0_4_2 = 0.0
e_0_5_0 = 0.0
e_0_5_1 = 1.0
e_0_5_2 = 0.0
e_0_6_0 = 0.0
e_0_6_1 = 0.0
e_0_6_2 = 0.0
e_0_7_0 = 0.0
e_0_7_1 = 0.0
e_0_7_2 = 0.0
e_0_8_0 = 0.0
e_0_8_1 = 0.0
e_0_8_2 = 0.0
e_0_9_0 = 0.0
e_0_9_1 = 0.0
e_0_9_2 = 0.0
e_1_2_0 = 0.0
e_1_2_1 = 0.0
e_1_2_2 = 0.0
e_1_3_0 = 0.0
e_1_3_1 = 0.0
e_1_3_2 = 0.0
e_1_4_0 = 0.0
e_1_4_1 = 0.0
e_1_4_2 = 0.0
e_1_5_0 = 0.0
e_1_5_1 = 0.0
e_1_5_2 = 0.0
e_1_6_0 = 0.0
e_1_6_1 = 0.0
e_1_6_2 = 0.0
e_1_7_0 = 0.0
e_1_7_1 = 0.0
e_1_7_2 = 1.0
e_1_8_0 = 0.0
e_1_8_1 = 0.0
e_1_8_2 = 0.0
e_1_9_0 = 0.0
e_1_9_1 = 0.0
e_1_9_2 = 1.0
e_2_3_0 = 0.0
e_2_3_1 = 0.0
e_2_3_2 = 0.0
e_2_4_0 = 0.0
e_2_4_1 = 0.0
e_2_4_2 = 0.0
e_2_5_0 = 0.0
e_2_5_1 = 1.0
e_2_5_2 = 0.0
e_2_6_0 = 0.0
e_2_6_1 = 0.0
e_2_6_2 = 0.0
e_2_7_0 = 0.0
e_2_7_1 = 0.0
e_2_7_2 = 0.0
e_2_8_0 = 0.0
e_2_8_1 = 0.0
e_2_8_2 = 0.0
e_2_9_0 = 0.0
e_2_9_1 = 0.0
e_2_9_

0=1, 5=1, 1=2, 7=2, 9=2, 3=0, 4=0, 6=0, 8=0, 

# Gurobi Infra 

In [2337]:
sample_input = pd.read_csv("test50.txt", header = None)
n = int(list(sample_input[0])[0])
# s_max = int(list(sample_input[0])[1])
s_max = 10000
K = 10

In [2338]:
# Parsing input data 
df = sample_input
df = pd.DataFrame([x.split() for x in df[0].tolist() ])
df = df.loc[2:,:]
df["i"] = df.loc[:,0]
df["j"] = df.loc[:,1]
df["sadness"] = df.loc[:,3]
df["happiness"] = df.loc[:,2]
df = df[["i", "j", "sadness", "happiness"]]

In [2339]:
i_id = [int(i) for i in list(df["i"])]
j_id = [int(i) for i in list(df["j"])]
sadness = [float(i) for i in list(df["sadness"])]*K
happiness = [float(i) for i in list(df["happiness"])]*K

sadness_1 = [float(i) for i in list(df["sadness"])]
happiness_1 = [float(i) for i in list(df["happiness"])]
len(sadness) == len(happiness)

True

In [2340]:
m = gp.Model("proj")

In [2341]:
d_e = {}
for k in range(K):
    for i in range(comb(n,2)):
        d_e["e_{0}_{1}_{2}".format(i_id[i],j_id[i],k)] = m.addVar(name= "e_{0}_{1}_{2}".format(i_id[i],j_id[i],k), vtype = GRB.BINARY)
edges_1 = list(d_e.values())

In [2342]:
# Create another edge array with different sort
edges_2 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[4]))]

# Create another edge array for constraint 4 
edges_3 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[6]))]

# Check
len(edges_1) == comb(n,2)*K

True

In [2343]:
# vertices 
d_v = {}
for j in range(K):
    for i in range(n):
        d_v["v_{0}_{1}".format(i,j)] = m.addVar(name = "v_{0}_{1}".format(i,j),  vtype = GRB.BINARY)
vertices_1 = list(d_v.values())

# Create another edge array with different sort
vertices_2 = [d_v.get(i) for i in sorted(d_v.keys(), key=lambda x: x[2])]

# Check
len(vertices_1) == n*K

True

In [2344]:
# Linearization of quadratic variables 1 
v_list = np.array(np.array_split(vertices_1, K)).tolist()
v_big = []
for i in range(K):
    j = 0
    while j != n:
        for _ in range(j,n-1):
            v_big.append(v_list[i][j])
        j += 1
        
print(len(edges_1) == len(v_big))

True


In [2345]:
# Linearization of quadratic variables 2
v_list_2 = [i[1:] for i in v_list]
v_big_2 = []
for h in range(K):
    j = 0
    while j != n:
        for i in range(j,n-1):
            v_big_2.append(v_list_2[h][i])
        j += 1
        i = j
print(len(edges_1) == len(v_big_2))

True


In [2346]:
m.setObjective(sum(edges_1[i] * happiness[i] for i in range(len(happiness))), GRB.MAXIMIZE)

In [2347]:
# # Not sure if we need this constraint now 
# # Constraint 1 
# constraint_2 = [sum(edges_1[i:i + K]) for i in range(0, len(edges_1), K)]
# for i in range(len(constraint_3)):
#     m.addConstr(constraint_2[i] == 1)

In [2348]:
aa = []
h = 0
for i in range(K):
    for j in range(comb(n,2)):
        aa.append(edges_1[h] * sadness_1[j])
        h += 1

constraint_2 = [sum(aa[i:i+ comb(n,2)]) for i in range(0, len(aa), comb(n,2))]
for i in range(len(constraint_2)):
    m.addConstr(constraint_2[i] <=  s_max / K)

In [2349]:
constraint_3 = [sum(vertices_2[i:i + K]) for i in range(0, len(vertices_2), K)]
for i in range(len(constraint_3)):
    m.addConstr(constraint_3[i] == 1)

In [2350]:
for i in range(len(edges_1)):
     m.addConstr(edges_1[i] <= v_big[i])
    
# Constraint 4.2
for i in range(len(edges_1)):
     m.addConstr(edges_1[i] <= v_big_2[i])


# Constraint 4.3
for i in range(len(edges_1)):
    m.addConstr(edges_1[i] >= v_big[i] + v_big_2[i] - 1)

# # Constraint 4'
# for i in range(len(edges_1)):
#     m.addConstr(edges_1[i] == v_big[i] * v_big_2[i])

In [2351]:
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 36810 rows, 12750 columns and 98500 nonzeros
Model fingerprint: 0x1bca3d36
Variable types: 0 continuous, 12750 integer (12750 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+02]
  Objective range  [2e-02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 4612.6130000
Presolve removed 3960 rows and 1320 columns
Presolve time: 1.99s
Presolved: 32850 rows, 11430 columns, 87940 nonzeros
Variable types: 0 continuous, 11430 integer (11430 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    9425    2.6382493e+04   5.894332e+01   0.000000e+00      5s
    9871    2.6376372e+04   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 2.637637e+04, 9871 iterations, 3.32 seconds

    Nodes    |    Current Node    |     

In [2353]:
for v in m.getVars():
    print((v.varName , v.x))

('e_0_1_0', -0.0)
('e_0_2_0', -0.0)
('e_0_3_0', -0.0)
('e_0_4_0', -0.0)
('e_0_5_0', -0.0)
('e_0_6_0', -0.0)
('e_0_7_0', -0.0)
('e_0_8_0', -0.0)
('e_0_9_0', -0.0)
('e_0_10_0', -0.0)
('e_0_11_0', -0.0)
('e_0_12_0', -0.0)
('e_0_13_0', -0.0)
('e_0_14_0', -0.0)
('e_0_15_0', -0.0)
('e_0_16_0', -0.0)
('e_0_17_0', -0.0)
('e_0_18_0', -0.0)
('e_0_19_0', -0.0)
('e_0_20_0', -0.0)
('e_0_21_0', -0.0)
('e_0_22_0', -0.0)
('e_0_23_0', -0.0)
('e_0_24_0', -0.0)
('e_0_25_0', -0.0)
('e_0_26_0', -0.0)
('e_0_27_0', -0.0)
('e_0_28_0', -0.0)
('e_0_29_0', -0.0)
('e_0_30_0', -0.0)
('e_0_31_0', -0.0)
('e_0_32_0', -0.0)
('e_0_33_0', -0.0)
('e_0_34_0', -0.0)
('e_0_35_0', -0.0)
('e_0_36_0', -0.0)
('e_0_37_0', -0.0)
('e_0_38_0', -0.0)
('e_0_39_0', -0.0)
('e_0_40_0', -0.0)
('e_0_41_0', -0.0)
('e_0_42_0', -0.0)
('e_0_43_0', -0.0)
('e_0_44_0', -0.0)
('e_0_45_0', -0.0)
('e_0_46_0', -0.0)
('e_0_47_0', -0.0)
('e_0_48_0', -0.0)
('e_0_49_0', -0.0)
('e_1_2_0', -0.0)
('e_1_3_0', -0.0)
('e_1_4_0', -0.0)
('e_1_5_0', -0.0)
('e_1_

('e_10_47_1', -0.0)
('e_10_48_1', -0.0)
('e_10_49_1', -0.0)
('e_11_12_1', 0.0)
('e_11_13_1', 0.0)
('e_11_14_1', 0.0)
('e_11_15_1', 0.0)
('e_11_16_1', 0.0)
('e_11_17_1', 0.0)
('e_11_18_1', -0.0)
('e_11_19_1', -0.0)
('e_11_20_1', -0.0)
('e_11_21_1', -0.0)
('e_11_22_1', -0.0)
('e_11_23_1', -0.0)
('e_11_24_1', -0.0)
('e_11_25_1', -0.0)
('e_11_26_1', -0.0)
('e_11_27_1', -0.0)
('e_11_28_1', -0.0)
('e_11_29_1', -0.0)
('e_11_30_1', -0.0)
('e_11_31_1', -0.0)
('e_11_32_1', -0.0)
('e_11_33_1', -0.0)
('e_11_34_1', -0.0)
('e_11_35_1', -0.0)
('e_11_36_1', -0.0)
('e_11_37_1', -0.0)
('e_11_38_1', -0.0)
('e_11_39_1', -0.0)
('e_11_40_1', -0.0)
('e_11_41_1', -0.0)
('e_11_42_1', -0.0)
('e_11_43_1', -0.0)
('e_11_44_1', -0.0)
('e_11_45_1', -0.0)
('e_11_46_1', -0.0)
('e_11_47_1', -0.0)
('e_11_48_1', -0.0)
('e_11_49_1', -0.0)
('e_12_13_1', 0.0)
('e_12_14_1', 0.0)
('e_12_15_1', 0.0)
('e_12_16_1', 0.0)
('e_12_17_1', 0.0)
('e_12_18_1', -0.0)
('e_12_19_1', -0.0)
('e_12_20_1', -0.0)
('e_12_21_1', -0.0)
('e_12_22_1

('e_8_42_2', -0.0)
('e_8_43_2', -0.0)
('e_8_44_2', -0.0)
('e_8_45_2', -0.0)
('e_8_46_2', -0.0)
('e_8_47_2', -0.0)
('e_8_48_2', -0.0)
('e_8_49_2', -0.0)
('e_9_10_2', -0.0)
('e_9_11_2', -0.0)
('e_9_12_2', -0.0)
('e_9_13_2', -0.0)
('e_9_14_2', -0.0)
('e_9_15_2', -0.0)
('e_9_16_2', -0.0)
('e_9_17_2', -0.0)
('e_9_18_2', -0.0)
('e_9_19_2', -0.0)
('e_9_20_2', -0.0)
('e_9_21_2', -0.0)
('e_9_22_2', -0.0)
('e_9_23_2', -0.0)
('e_9_24_2', -0.0)
('e_9_25_2', -0.0)
('e_9_26_2', -0.0)
('e_9_27_2', -0.0)
('e_9_28_2', -0.0)
('e_9_29_2', -0.0)
('e_9_30_2', -0.0)
('e_9_31_2', -0.0)
('e_9_32_2', -0.0)
('e_9_33_2', -0.0)
('e_9_34_2', -0.0)
('e_9_35_2', -0.0)
('e_9_36_2', -0.0)
('e_9_37_2', -0.0)
('e_9_38_2', -0.0)
('e_9_39_2', -0.0)
('e_9_40_2', -0.0)
('e_9_41_2', -0.0)
('e_9_42_2', -0.0)
('e_9_43_2', -0.0)
('e_9_44_2', -0.0)
('e_9_45_2', -0.0)
('e_9_46_2', -0.0)
('e_9_47_2', -0.0)
('e_9_48_2', -0.0)
('e_9_49_2', -0.0)
('e_10_11_2', 0.0)
('e_10_12_2', 0.0)
('e_10_13_2', 0.0)
('e_10_14_2', 0.0)
('e_10_15_2'

('e_16_25_3', -0.0)
('e_16_26_3', -0.0)
('e_16_27_3', -0.0)
('e_16_28_3', -0.0)
('e_16_29_3', -0.0)
('e_16_30_3', -0.0)
('e_16_31_3', -0.0)
('e_16_32_3', -0.0)
('e_16_33_3', -0.0)
('e_16_34_3', -0.0)
('e_16_35_3', -0.0)
('e_16_36_3', -0.0)
('e_16_37_3', -0.0)
('e_16_38_3', -0.0)
('e_16_39_3', -0.0)
('e_16_40_3', -0.0)
('e_16_41_3', -0.0)
('e_16_42_3', -0.0)
('e_16_43_3', -0.0)
('e_16_44_3', -0.0)
('e_16_45_3', -0.0)
('e_16_46_3', -0.0)
('e_16_47_3', -0.0)
('e_16_48_3', -0.0)
('e_16_49_3', -0.0)
('e_17_18_3', 0.0)
('e_17_19_3', 0.0)
('e_17_20_3', -0.0)
('e_17_21_3', -0.0)
('e_17_22_3', -0.0)
('e_17_23_3', -0.0)
('e_17_24_3', -0.0)
('e_17_25_3', -0.0)
('e_17_26_3', -0.0)
('e_17_27_3', -0.0)
('e_17_28_3', -0.0)
('e_17_29_3', -0.0)
('e_17_30_3', -0.0)
('e_17_31_3', -0.0)
('e_17_32_3', -0.0)
('e_17_33_3', -0.0)
('e_17_34_3', -0.0)
('e_17_35_3', -0.0)
('e_17_36_3', -0.0)
('e_17_37_3', -0.0)
('e_17_38_3', -0.0)
('e_17_39_3', -0.0)
('e_17_40_3', -0.0)
('e_17_41_3', -0.0)
('e_17_42_3', -0.0)
('

('e_10_44_4', -0.0)
('e_10_45_4', -0.0)
('e_10_46_4', -0.0)
('e_10_47_4', -0.0)
('e_10_48_4', -0.0)
('e_10_49_4', -0.0)
('e_11_12_4', 0.0)
('e_11_13_4', 0.0)
('e_11_14_4', 0.0)
('e_11_15_4', -0.0)
('e_11_16_4', -0.0)
('e_11_17_4', -0.0)
('e_11_18_4', -0.0)
('e_11_19_4', -0.0)
('e_11_20_4', -0.0)
('e_11_21_4', -0.0)
('e_11_22_4', -0.0)
('e_11_23_4', -0.0)
('e_11_24_4', -0.0)
('e_11_25_4', -0.0)
('e_11_26_4', -0.0)
('e_11_27_4', -0.0)
('e_11_28_4', -0.0)
('e_11_29_4', -0.0)
('e_11_30_4', -0.0)
('e_11_31_4', -0.0)
('e_11_32_4', -0.0)
('e_11_33_4', -0.0)
('e_11_34_4', -0.0)
('e_11_35_4', -0.0)
('e_11_36_4', -0.0)
('e_11_37_4', -0.0)
('e_11_38_4', -0.0)
('e_11_39_4', -0.0)
('e_11_40_4', -0.0)
('e_11_41_4', -0.0)
('e_11_42_4', -0.0)
('e_11_43_4', -0.0)
('e_11_44_4', -0.0)
('e_11_45_4', -0.0)
('e_11_46_4', -0.0)
('e_11_47_4', -0.0)
('e_11_48_4', -0.0)
('e_11_49_4', -0.0)
('e_12_13_4', 0.0)
('e_12_14_4', 0.0)
('e_12_15_4', -0.0)
('e_12_16_4', -0.0)
('e_12_17_4', -0.0)
('e_12_18_4', -0.0)
('e_1

('e_17_42_5', -0.0)
('e_17_43_5', -0.0)
('e_17_44_5', -0.0)
('e_17_45_5', -0.0)
('e_17_46_5', -0.0)
('e_17_47_5', -0.0)
('e_17_48_5', -0.0)
('e_17_49_5', -0.0)
('e_18_19_5', 0.0)
('e_18_20_5', -0.0)
('e_18_21_5', -0.0)
('e_18_22_5', -0.0)
('e_18_23_5', -0.0)
('e_18_24_5', -0.0)
('e_18_25_5', -0.0)
('e_18_26_5', -0.0)
('e_18_27_5', -0.0)
('e_18_28_5', -0.0)
('e_18_29_5', -0.0)
('e_18_30_5', -0.0)
('e_18_31_5', -0.0)
('e_18_32_5', -0.0)
('e_18_33_5', -0.0)
('e_18_34_5', -0.0)
('e_18_35_5', -0.0)
('e_18_36_5', -0.0)
('e_18_37_5', -0.0)
('e_18_38_5', -0.0)
('e_18_39_5', -0.0)
('e_18_40_5', -0.0)
('e_18_41_5', -0.0)
('e_18_42_5', -0.0)
('e_18_43_5', -0.0)
('e_18_44_5', -0.0)
('e_18_45_5', -0.0)
('e_18_46_5', -0.0)
('e_18_47_5', -0.0)
('e_18_48_5', -0.0)
('e_18_49_5', -0.0)
('e_19_20_5', -0.0)
('e_19_21_5', -0.0)
('e_19_22_5', -0.0)
('e_19_23_5', -0.0)
('e_19_24_5', -0.0)
('e_19_25_5', -0.0)
('e_19_26_5', -0.0)
('e_19_27_5', -0.0)
('e_19_28_5', -0.0)
('e_19_29_5', -0.0)
('e_19_30_5', -0.0)
(

('e_28_30_6', -0.0)
('e_28_31_6', -0.0)
('e_28_32_6', -0.0)
('e_28_33_6', -0.0)
('e_28_34_6', -0.0)
('e_28_35_6', -0.0)
('e_28_36_6', -0.0)
('e_28_37_6', -0.0)
('e_28_38_6', -0.0)
('e_28_39_6', -0.0)
('e_28_40_6', -0.0)
('e_28_41_6', -0.0)
('e_28_42_6', -0.0)
('e_28_43_6', -0.0)
('e_28_44_6', -0.0)
('e_28_45_6', -0.0)
('e_28_46_6', -0.0)
('e_28_47_6', -0.0)
('e_28_48_6', -0.0)
('e_28_49_6', -0.0)
('e_29_30_6', -0.0)
('e_29_31_6', -0.0)
('e_29_32_6', -0.0)
('e_29_33_6', -0.0)
('e_29_34_6', -0.0)
('e_29_35_6', -0.0)
('e_29_36_6', -0.0)
('e_29_37_6', -0.0)
('e_29_38_6', -0.0)
('e_29_39_6', -0.0)
('e_29_40_6', -0.0)
('e_29_41_6', -0.0)
('e_29_42_6', -0.0)
('e_29_43_6', -0.0)
('e_29_44_6', -0.0)
('e_29_45_6', -0.0)
('e_29_46_6', -0.0)
('e_29_47_6', -0.0)
('e_29_48_6', -0.0)
('e_29_49_6', -0.0)
('e_30_31_6', 0.0)
('e_30_32_6', 0.0)
('e_30_33_6', -0.0)
('e_30_34_6', -0.0)
('e_30_35_6', -0.0)
('e_30_36_6', -0.0)
('e_30_37_6', -0.0)
('e_30_38_6', -0.0)
('e_30_39_6', -0.0)
('e_30_40_6', -0.0)
('

('e_0_46_8', -0.0)
('e_0_47_8', -0.0)
('e_0_48_8', -0.0)
('e_0_49_8', -0.0)
('e_1_2_8', -0.0)
('e_1_3_8', 1.0)
('e_1_4_8', -0.0)
('e_1_5_8', -0.0)
('e_1_6_8', -0.0)
('e_1_7_8', -0.0)
('e_1_8_8', 1.0)
('e_1_9_8', -0.0)
('e_1_10_8', 0.0)
('e_1_11_8', -0.0)
('e_1_12_8', -0.0)
('e_1_13_8', 1.0)
('e_1_14_8', -0.0)
('e_1_15_8', -0.0)
('e_1_16_8', -0.0)
('e_1_17_8', -0.0)
('e_1_18_8', -0.0)
('e_1_19_8', -0.0)
('e_1_20_8', -0.0)
('e_1_21_8', -0.0)
('e_1_22_8', -0.0)
('e_1_23_8', -0.0)
('e_1_24_8', -0.0)
('e_1_25_8', -0.0)
('e_1_26_8', 1.0)
('e_1_27_8', -0.0)
('e_1_28_8', -0.0)
('e_1_29_8', -0.0)
('e_1_30_8', -0.0)
('e_1_31_8', -0.0)
('e_1_32_8', -0.0)
('e_1_33_8', -0.0)
('e_1_34_8', -0.0)
('e_1_35_8', -0.0)
('e_1_36_8', -0.0)
('e_1_37_8', -0.0)
('e_1_38_8', -0.0)
('e_1_39_8', 1.0)
('e_1_40_8', -0.0)
('e_1_41_8', -0.0)
('e_1_42_8', -0.0)
('e_1_43_8', -0.0)
('e_1_44_8', 1.0)
('e_1_45_8', -0.0)
('e_1_46_8', -0.0)
('e_1_47_8', -0.0)
('e_1_48_8', -0.0)
('e_1_49_8', -0.0)
('e_2_3_8', -0.0)
('e_2_4_8

('e_6_47_9', -0.0)
('e_6_48_9', -0.0)
('e_6_49_9', -0.0)
('e_7_8_9', -0.0)
('e_7_9_9', 1.0)
('e_7_10_9', -0.0)
('e_7_11_9', 1.0)
('e_7_12_9', -0.0)
('e_7_13_9', -0.0)
('e_7_14_9', -0.0)
('e_7_15_9', -0.0)
('e_7_16_9', -0.0)
('e_7_17_9', -0.0)
('e_7_18_9', -0.0)
('e_7_19_9', -0.0)
('e_7_20_9', -0.0)
('e_7_21_9', -0.0)
('e_7_22_9', -0.0)
('e_7_23_9', -0.0)
('e_7_24_9', -0.0)
('e_7_25_9', -0.0)
('e_7_26_9', 1.0)
('e_7_27_9', -0.0)
('e_7_28_9', -0.0)
('e_7_29_9', -0.0)
('e_7_30_9', -0.0)
('e_7_31_9', -0.0)
('e_7_32_9', -0.0)
('e_7_33_9', -0.0)
('e_7_34_9', -0.0)
('e_7_35_9', 1.0)
('e_7_36_9', -0.0)
('e_7_37_9', -0.0)
('e_7_38_9', -0.0)
('e_7_39_9', -0.0)
('e_7_40_9', -0.0)
('e_7_41_9', -0.0)
('e_7_42_9', -0.0)
('e_7_43_9', -0.0)
('e_7_44_9', -0.0)
('e_7_45_9', 1.0)
('e_7_46_9', -0.0)
('e_7_47_9', -0.0)
('e_7_48_9', -0.0)
('e_7_49_9', -0.0)
('e_8_9_9', -0.0)
('e_8_10_9', -0.0)
('e_8_11_9', -0.0)
('e_8_12_9', -0.0)
('e_8_13_9', -0.0)
('e_8_14_9', -0.0)
('e_8_15_9', -0.0)
('e_8_16_9', -0.0)
(